# 『人を知る』人工知能講座 <br> <span style="color: #00B0F0;">Session 3 言語メディア</span> <br> <span style="background-color: #1F4E79; color: #FFFFFF;">&nbsp;3&nbsp;</span> BERTによる自然言語処理 〜日本語Fine-tuning〜 

日本語では英語のGLUEデータセットのように大規模な種々のデータセットが整備されているわけではありませんので、タグ付きコーパスからデータセットを生成し、fine-tuningします。

本演習では1日目で用いたKNBC (Kyoto University and NTT Blog Corpus)を用います。データは `/data/nlp/text/KNBC_v1.0_090925_utf8` にあります。

In [1]:
!ls -l /data/nlp/text/KNBC_v1.0_090925_utf8

total 1276
-rw-r--r--   1 root root   5915  9月 30  2009 README.txt
drwxr-xr-x   2 root root   4096 11月 12 13:04 bin
drwxr-xr-x 251 root root  12288 11月 12 13:04 corpus1
drwxr-xr-x   2 root root   4096 11月 12 13:04 corpus2
drwxr-xr-x   2 root root 503808 11月 12 13:04 html
-rw-r--r--   1 root root 770764  9月 25  2009 manual.pdf


1日目と同様、アノテーションを可視化できる html を確認してみましょう。

In [2]:
from IPython.display import HTML

KNBC_dir = "/data/nlp/text/KNBC_v1.0_090925_utf8"
HTML("<style type='text/css'>" + open(f"{KNBC_dir}/html/knbc_article_index.css").read() + "</style>")
HTML(open(f"{KNBC_dir}/html/KN001_Kyoto_1-1-17-01.html").read())

係り受け,格・省略・照応、固有表現,評判表現
時雨殿の┓,"時雨殿:=:2文前, 時雨殿:LOCA",
外見も━━━━┓,時雨殿:ノ,時雨殿:批評＋
周りの┓ ┃,時雨殿:ノ,
料亭に┓ ┃,"周り:修飾, 料亭:=≒:1文前",
負けていない┓┃,"外見:ガ, 料亭:ニ",
程┫,,
’高級’である。,外見:ガ,


このコーパスを使って、以下のタスクをfine-tuningで解きます。

* 評判分析
* 文書分類
* 固有表現解析

## 1. 評判分析

GLUEの最初で見た SST-2 データセットと全く同じ形式のファイルを生成し、fine-tuningのコマンドを動かします。

再掲: KNBCでは「批評」(賛成と反対)や「感情」(好きと嫌い)など、いくつかの軸について、評判を保持している人や評判の対象などが付与されています。ここではfine-tuningを動かすことを目的としますので、かなり荒っぽいですが、「批評」の軸について賛成(「批評＋」と表記されている)を表す表現があれば文全体をpositive、反対(同様に「批評−」)があれば文全体をnegativeとみなすことにします。

### 手順1. 前処理

上記の基準をもとに以下のpythonスクリプトで評判分析のデータセットを生成します。1日目と同じスクリプトです。

In [3]:
import glob
import os
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def get_words(sid):
    # sid: KN001_Keitai_1-1-12-01
    # -> KNBC_v1.0_090925/corpus1/KN001_Keitai_1/KN001_Keitai_1-1-12-01
    subdirname = (sid.split("-"))[0]
    filename = f"{KNBC_dir}/corpus1/{subdirname}/{sid}"
    if os.path.exists(filename) is False:
        return None

    # 単語集合を得る (正解単語区切を利用)
    words = []
    with open(filename, "r", encoding="utf-8") as reader:
        buf = ""
        for line in reader.readlines():
            if line.startswith(("#", "*", "+", "EOS")):
                continue
            word = (line.split(" "))[0]
            words.append(word)

    return words

def get_sentiment_label(sentiment_type):
    # 批評＋を含めば1, 批評−を含めば0
    # 両方含むものはスキップ
    if "批評＋" in sentiment_type and "批評−" in sentiment_type:
        return None
    if "批評＋" in sentiment_type:
        return 1
    if "批評−" in sentiment_type:
        return 0
    return None

out_dir = "sentiment_analysis_KNBC"
os.makedirs(out_dir, exist_ok=True)
f_out = open(f"{out_dir}/all.tsv", "w")

for filename in glob.glob(f"{KNBC_dir}/corpus2/*"):
    with open(filename, "r", encoding="utf-8") as reader:
        buf = ""
        for line in reader.readlines():
            # 文ID, 文, 評判保持者, 評判表現, 評判タイプ, 評判対象
            # KN001_Keitai_1-1-12-01	確かにプリペイドには、いくつかの弱点がある。	[著者]	いくつかの弱点がある	批評−	プリペイド
            sid, sentence, _, _, sentiment_type, _ = line.strip("\n").split("\t")
            words = get_words(sid)
            if words is None:
              # logger.warning(f"skip: {sid} {sentence}")
              continue
            label = get_sentiment_label(sentiment_type)
            tokenized_sentence = " ".join(words)
            if label is not None:         
                print(f"{tokenized_sentence}\t{label}", file=f_out)

f_out.close()

print("完了!")
!wc -l sentiment_analysis_KNBC/*

完了!
680 sentiment_analysis_KNBC/all.tsv


680文生成されました。

データの中身を見てみます。以下ではランダムに選んだ10件を表形式で表示しています。

In [4]:
import pandas as pd

data_sentiment = pd.read_csv('sentiment_analysis_KNBC/all.tsv', encoding='utf-8', delimiter='\t', names=('sentence', 'label'))

data_sentiment.sample(10).style.set_table_styles(
                [{'selector': 'th',
                  'props': [('text-align', 'center')]}, 
                 {'selector': 'td',
                  'props': [('text-align', 'left')]}])

,sentence,label
391,塩 タン に カルビ に ハラミ に 、 そこ は まるで 焼肉 パラダイス ☆☆,1
0,今 の １ 回生 で は １ 番 漕 暦 が 浅い のに 持ち前 の 体力 と 精神 力 で レース 出場 権 を 手 に した 。,1
344,以外に おいしかった よー 。,1
3,今年 の インカレ で 男子 エイト が 準 優勝 する くらい 、 男子 は 部員 も 多く 、 練習 も きつい のに 、 私 たち は 彼ら より も 多く の 練習 を こなして きた 。,1
446,前 に だれ か が いって た けど 、 外国 人 に は やっぱ 金閣寺 の ほう が 感激 の 度合い が 大きい らしい です 。,1
293,誰 でも 割り が 始まって しまった から 、 料金 の 比較 も ぁんまり 頼り に なら なくて 。,0
387,… つくづく 幸せな 人間 である 。,1
4,いける かも しれ ない 。,1
201,とても 安心 して 使える もの だ と いう のだ 。,1
535,どうせ 、 観光 客 は 殆ど が 次に 挙げる ような 美辞麗句 だけ 鵜呑み に して いる のだ 。,0


pandasを使えば以下のようにラベルの頻度を簡単に集計することができます。"1"が"0"の2倍弱多いことがわかります。

In [5]:
data_sentiment["label"].value_counts()

1    436
0    244
Name: label, dtype: int64

次にtrain/dev/testに分割 (split)します。ここでは8:1:1の割合で分割します。ここでもpandasを使います。

In [6]:
 import numpy as np
 
 def split_train_dev_test(data, dirname, header,
                         split_ratios=(0.8, 0.1, 0.1)):

    ts = data.shape
    df = pd.DataFrame(data)
    shuffle_df = df.reindex(np.random.permutation(df.index))

    indice_1 = int(ts[0] * split_ratios[0])
    indice_2 = int(ts[0] * (split_ratios[0] + split_ratios[1]))

    shuffle_df[:indice_1].to_csv(f"{dirname}/train.tsv",header=header, sep='\t', index=False)
    shuffle_df[indice_1:indice_2].to_csv(f"{dirname}/dev.tsv",header=header, sep='\t', index=False)
    shuffle_df[indice_2:].to_csv(f"{dirname}/test.tsv",header=header, sep='\t', index=False)        

In [7]:
split_train_dev_test(data_sentiment, "sentiment_analysis_KNBC", ("sentence", "label"))
print("完了!")

完了!


以下のように分割されました。

In [8]:
!wc -l sentiment_analysis_KNBC/*.tsv

   680 sentiment_analysis_KNBC/all.tsv
    69 sentiment_analysis_KNBC/dev.tsv
    69 sentiment_analysis_KNBC/test.tsv
   545 sentiment_analysis_KNBC/train.tsv
  1363 total


中身をみてみます。SST-2と同じ形式のファイルができました。
実際のデータは形態素分析をして、入力してあげる必要がある

In [9]:
!head sentiment_analysis_KNBC/train.tsv

sentence	label
私 自身 の 携帯 に は 装備 さ れて い なかった ので ， これ が 結構 大きな 買い換え に 対する 動機付け と なり ました （ 笑 ）	1
そして 、 その 祭り が どれ も 、 それなり に 歴史 なり 、 由来 が ある から 恐ろしい 。	1
前 に だれ か が いって た けど 、 外国 人 に は やっぱ 金閣寺 の ほう が 感激 の 度合い が 大きい らしい です 。	1
なんだか 支離滅裂な 文章 に なって しまい ました が 今回 は これ で おしまい 。	0
つまり 「 無い物ねだり 」 である 。	0
確かに プリペイド に は 、 いく つ か の 弱点 が ある 。	0
面倒やん 。	0
確かに 銀閣寺 は しょぼい 。	0
デジカメ が ない 時 でも ぱっと 撮影 できて 大変 便利である 。	1


### 手順2. Fine-tuning

英語のGLUEのfine-tuningと同様に run_glue.py を実行します。変更点は以下のとおりです。
*   --model_name_or_pathオプションで日本語pre-trainedモデルを指定 (/data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers)
*   --do_lower_caseを削除 (tokenizer_config.jsonでfalseにしていますが念のため)
*   --data_dirオプションで先ほど作ったデータのディレクトリを指定

SST-2よりもさらにサイズが小さいため、3エポックでも2分程度で終わります。


In [10]:
!python ./transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
    --task_name "SST-2" \
    --do_train \
    --do_eval \
    --save_steps 1000 \
    --data_dir sentiment_analysis_KNBC \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --output_dir KNBC_result/sentiment_analysis/ \
    --overwrite_output_dir \
    --overwrite_cache

11/28/2019 15:23:44 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/28/2019 15:23:44 - INFO - transformers.configuration_utils -   loading configuration file /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/config.json
11/28/2019 15:23:44 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 32006
}

11/28/2019 15:23:44 - INFO - transforme

11/28/2019 15:23:50 - INFO - __main__ -   Saving features into cached file sentiment_analysis_KNBC/cached_train_Japanese_L-12_H-768_A-12_E-30_BPE_transformers_128_sst-2
11/28/2019 15:23:50 - INFO - __main__ -   ***** Running training *****
11/28/2019 15:23:50 - INFO - __main__ -     Num examples = 544
11/28/2019 15:23:50 - INFO - __main__ -     Num Epochs = 3
11/28/2019 15:23:50 - INFO - __main__ -     Instantaneous batch size per GPU = 8
11/28/2019 15:23:50 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 8
11/28/2019 15:23:50 - INFO - __main__ -     Gradient Accumulation steps = 1
11/28/2019 15:23:50 - INFO - __main__ -     Total optimization steps = 204
Iteration:  26%|████████▍                       | 18/68 [00:08<00:22,  2.19it/s]


Iteration:  68%|█████████████████████▋          | 46/68 [00:20<00:10,  2.18it/s]


Epoch: 100%|██████████████████████████████████████| 3/3 [01:32<00:00, 30.99s/it]
11/28/2019 15:25:23 - INFO - __main__ -    global_step = 204, average loss = 0.3245756413203244
11/28/2019 15:25:23 - INFO - __main__ -   Saving model checkpoint to KNBC_result/sentiment_analysis/
11/28/2019 15:25:23 - INFO - transformers.configuration_utils -   Configuration saved in KNBC_result/sentiment_analysis/config.json
11/28/2019 15:25:23 - INFO - transformers.modeling_utils -   Model weights saved in KNBC_result/sentiment_analysis/pytorch_model.bin
11/28/2019 15:25:23 - INFO - transformers.configuration_utils -   loading configuration file KNBC_result/sentiment_analysis/config.json
11/28/2019 15:25:23 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_

Evaluating: 100%|█████████████████████████████████| 9/9 [00:01<00:00,  8.16it/s]
11/28/2019 15:25:30 - INFO - __main__ -   ***** Eval results  *****
11/28/2019 15:25:30 - INFO - __main__ -     acc = 0.8235294117647058


一番最後の数字がdevにおける精度 (accuracy)で、80%前後になると思います。1日目のLSTMによるモデルでは精度70%前後でしたので、BERTがいかに強力であるかがおわかりかと思います。

一般に、ニューラルネットワークのモデルで高い精度を達成するために大量のトレーニングデータが必要と言われます。しかし、BERTはBi-LSTMのようなこれまでのニューラルネットワークに比べるとそれほどトレーニングデータを必要としません。これは転移学習 (transfer learning)のおかげです。

### 手順3. 予測

次に、好きな文を入力し、予測 (prediction)してみましょう。まず、上記で学習したモデルを別のディレクトリにコピーしておきましょう。

In [11]:
!mkdir -p my_result
!cp -pr KNBC_result/sentiment_analysis/ my_result/

上記で使用したデータも別のディレクトリにコピーしましょう。

In [12]:
!cp -r sentiment_analysis_KNBC my_sentiment_analysis

そして、以下のようにしてdev.tsvを好きな文で上書きしてください。ここではわかち書きは手動で行ってください。ラベルは使いませんが0にしておいてください。

In [13]:
!echo -e 'sentence\tlabel\n朝 は いつも 眠い もの だ。\t0' > my_sentiment_analysis/dev.tsv
!head my_sentiment_analysis/dev.tsv

sentence	label
朝 は いつも 眠い もの だ。	0


以下のコマンドで予測します。先ほどとの違いは以下です。
* --do_train オプションを削除
* --save_steps 1000を削除
* --data_dir,  --output_dirオプションで今作ったディレクトリを指定

In [14]:
!python ./transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
    --task_name "SST-2" \
    --do_eval \
    --save_steps 1000 \
    --data_dir my_sentiment_analysis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --output_dir my_result/sentiment_analysis/ \
    --overwrite_output_dir \
    --overwrite_cache
!cat my_result/sentiment_analysis/dev_predictions.txt

11/28/2019 15:34:09 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/28/2019 15:34:09 - INFO - transformers.configuration_utils -   loading configuration file /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/config.json
11/28/2019 15:34:09 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 32006
}

11/28/2019 15:34:09 - INFO - transforme

Evaluating: 100%|█████████████████████████████████| 1/1 [00:00<00:00,  6.43it/s]
11/28/2019 15:34:17 - INFO - __main__ -   ***** Eval results  *****
11/28/2019 15:34:17 - INFO - __main__ -     acc = 0.0
朝 は いつも 眠い もの だ。	1


結果が一番下に出ます。いかがでしたでしょうか。

## 2. 文書分類

次に文書分類タスクを行います。先ほどの評判分析は1か0の2ラベルでしたが、今度は4カテゴリです。また、入力は1文ではなく文章になりますが、単に全部連結したものを入力とし、1文と同様に扱います。
※[SEP]を付ける場合は文同士が関係ないとき。

### 手順1. 前処理

In [15]:
out_dir = "text_classification_KNBC"
os.makedirs(out_dir, exist_ok=True)
labels = [ "Keitai", "Kyoto", "Gourmet", "Sports"]

def get_label(basename):
    for label in labels:
        if label in basename:
            return label
    else:
        return None

f_out = open(f"{out_dir}/all.tsv", "w")

for dir in glob.glob(f"{KNBC_dir}/corpus1/*"):
    basename = os.path.basename(dir)
    words = []
    
    # for each sentence
    sentence_index = 1
    while (True):
        filename = f"{dir}/{basename}-1-{sentence_index}-01"
        if os.path.exists(filename) is False:
            break

        header = True
        with open(filename, "r", encoding="utf-8") as reader:
            buf = ""
            for line in reader.readlines():
                if line.startswith(("#", "*", "+", "EOS")):
                    continue
                word = (line.split(" "))[0]
                
                # skip: category (例: [京都観光])
                if sentence_index == 1 and header is True:
                    continue
                if header is True and word == "]":
                    header = False
                words.append(word)
        sentence_index += 1

    label = get_label(basename)
    if label is not None:
        print("{}\t{}".format(" ".join(words), label), file=f_out)
    
f_out.close()
print("完了!")

完了!


しばらくは評判分析と同様のコマンドですので、どんどんコマンドを実行しながら内容を確認してください。

In [16]:
import pandas as pd
data_text_classification = pd.read_csv('text_classification_KNBC/all.tsv', encoding='utf-8', delimiter='\t', names=('sentence', 'label'))
data_text_classification.sample(10).style.set_table_styles(
                [{'selector': 'th',
                  'props': [('text-align', 'center')]}, 
                 {'selector': 'td',
                  'props': [('text-align', 'left')]}])

,sentence,label
199,私 は おそらく グルメ と は 対極 に ある 。 と いって も 食べる こと は 好きだ し 、 食べる 量 も 多い ほう だ と 思う 。 なに が グルメ で ない か と いう と 、 いわゆる 「 味 オンチ 」 だ と いう こと である 。 自分 で は そんな つもり は ぜんぜん ない のだ けど 、 人 が 「 これ は ちょっと ・・ 」 と いう もの も 平気で 平らげて しまう のだ 。 ３ 人 兄弟 の 末っ子 に 育った 環境 が そう さ せた の か は わから ない が 、 私 の 胃腸 の 強 さ は 相当な もの である らしい 。 とはいえ 、 自分 で は 「 おいしい もの 」 は 分かって いる つもりで いる 。 ときどき 実家 に 帰る と 「 あー なんて 自分 の 作る もの と は 違う んだろう 」 と へこむ し 、 お 金 を もった 社会 人 の 人 に おごって もらったり する と あまり の おいし さ に 感動 して しまう 。 グルメ を 自称 する 人 たち の おいし さ を 追及 する 努力 に は 感心 する けれど 、 なんでも にこにこ 食べ られる と いう の は 私 の 武器 だ と 思って いる 。 まあ 呆れ られる こと も 多い です が 。,Gourmet
167,先週 の 土曜日 に 部活 の コーチ と 友達 と で 銀閣寺 に いった 。 入り口 まで いった けど 俺 は そこ で 「 大文字 に 上ったら めっちゃ いい 景色 み れ ます 」 と いった 。 コーチ が 元気な ひと やった ので 、 じゃあ 大文字 上ろう と いう こと に なった 。 銀閣寺 に 入ら ず に 左 に 曲がる と 入り口 が あって 約 ３０ 分 で 上 まで 上れる 。 俺 は 大文字 が すきで 大学 に はいって から １０ 回 くらい は 上って る 。 今回 は 約 ２ ヶ月 ぶり 。 ３０ 分 かけて 上った 。 いつも 通り すごい 景色 が よかった 。 季節 に よって 少し 雰囲気 が 違う 。 秋 から 冬 に かけて は なんとなく 街 は どんより して いる 。 けど きらいじゃ ない どんより 具合 。 今回 も よい 景色 やった 。 おりて から コーチ は 銀閣寺 に 。 俺 は 一 度 入った こと が あった から 、 今回 は 遠慮 し といた 。,Kyoto
185,昨日 、 服 を 買い に 四 条 に 行った 。 物凄い 、 観光 客 と いう か 修学 旅行 客 が 歩いて た 。 新 京極 が 修学 旅行 客 で 埋めつくさ れて いた 。 明らかに 我が物顔 で 歩いて た 。 結構 ムカついた 。 いや 、 四 条 オレ の 街 だ から と か いって 小 一 時間 くらい 説教 し たく なる くらい だった 。 でも 女子 高 生 の ミニ スカート は 例え 修学 旅行 生 でも イイ もの だ ね 。 って 何 を 書いて んだ 、 オレ は ……… 。,Kyoto
116,今週 某 日 ， 授業 が かなり 早く 終わり ました ． そこ で ， 真っ直ぐ 帰宅 する の も 儚い ので ， 出町柳 から 丸太 町 まで 鴨川 沿い を 歩いて み ました ． スポーツ を して いる 人 や 散歩 して いる 人 ， 昼寝 を する 人 など ， 日常 の 京都 人 の 姿 を 拝見 でき ました ． 水量 も 前日 雨 が 降った わりに は 少なく ， 所々 に ある 川 の 岸 と 岸 を 結ぶ 石 を 飛び越える の も 容易でした ． 一部 の 石 は 亀 の 形 を して いて ， その 上 ちょっと ルート から 外れた ところ に も 亀 の 石 が あって ， 流れ の 勢い を 変える の に 必要な の か ？ と 疑問 に 思い ました ． 丸太 町 に 着いて ， 丸太 町 通 を 西 へ しばらく 進む と 河原 町 通 に ぶつかり ます ． この 近く に とっても おいしい ケーキ 屋 が あって ， 立ち寄って み ました ． ケーキ だけ で は なく ， クッキー など の 洋菓子 ， さらに は サンドイッチ も 置いて いて なかなか 品揃え が 豊富 ！ 奥 に カフェ コーナー が ある ので ， 紅茶 と ケーキ を 食べて み ました ． 味 は なかなか よかった し ， 店 内 の インテリア も なかなか 凝って い ました ． 少し 休憩 の つもり だった のに ， ２ 時間 も 居座って しまえる ほど 快適な 店 でした ． ちなみに その お 店 と は Ｓａｌｏｎ ｄｅ Ｔｈｅ Ｃｌｉｅｎｔｅｌｅ です ．,Gourmet
184,携帯 電話 って 便利だ よ ねー と か もはや 誰 も 言わ なく なった 。 だって 小学生 も 持って る んだ よ 、 この ご 時世 。 まさに ゆりかご から 墓場 まで に なり つつ ある 。 でも この 携帯 、 投げ出し たく なる 時 って ない ？ 俺 は あり ます よ 、 結構 。 まず さ 、 とりあえず 一 人 に なり たい とき って ない ？ そんな 気分 に なって 部屋 で ぼーっと して る と 着信 が 。。。 で 、 それ も で なきゃ いけない 相手 だったら 最悪 。 気分 が 害さ れて 、 結局 一 人 で 居 れ ない こと を 実感 。 あと は 、 旅行 で バカンス に 行って る とき ！！ こっち は さ 、 非 日常 ワールド を 体験 し たい んだ よ ！ 浸り たい んだ よ ！！ それ が 友達 でも バイト 関係 でも 電話 かかって くる と 、 もう さ 、 台無しだ よ ね 。 こちとら 貴重な 休暇 を 削って 来て る のに ！ ← 大 ウソ そんな わけ で 、 携帯 を 一 回 解約 しよう か な って 思い ました 。 多分 し ない んだ けど ね 。,Keitai
11,１０ 月 １４ 日 。 それ は １ 回生 の 私 と もう 一 人 、 １ 回生 女子 で 組んだ ダブルスカル と いう 、 ２ 人 乗り ボート の レース が あった 日 だ 。 １ 回生 同士 で 組んだ クルー で の 出場 は 初めて 。 天候 は 晴れ 。 やや 肌寒い くらい 。 ウォーミングアップ の ため 、 水上 へ 。 私 たち の 艇 は それなり に 艇 速 が 出て いた 。 対戦 相手 は 滋賀 教育 学部 ２ 艇 と 近畿 大学 。 １ 回生 で 編成 さ れた クルー は 私 たち だけ かも しれ ない 。 でも そんな の 関係 ない 。 やる しか ない 。 ウォーミングアップ の 安定 感 は 悪く ない 。 艇 速 も 出て いる 。 いける かも ・・・ ？ でも 自負 は だめ 。 油断 禁物 。 あと レース スタート まで １０ 分 。 スタート 地点 まで 行こう 。 ５ 分 前 ・・・ ３ 分 前 ・・・ ２ 分 前 ・・・ 「 滋賀 教育 学部 Ａ 、 近畿 大学 、 京都 大学 、 滋賀 教育 学部 Ｂ ・・・・ アテンション 、 ゴゥッ ！ 」 私 たち は スタート した 。 そんなに 悪く は ない スタート だった けど だめだ 。 少し 出遅れた 。 でも 私 たち の 持ち味 は レース 中盤 だ 。 １０００ メートル も ある し 大丈夫 ！！ まだ いける ！！ ３００ メートル 地点 。 私 たち は 追い上げた 。 滋賀 教育 学部 Ａ 、 Ｂ は 私 たち の 目の前 に いる から もう 大丈夫だ 。 でも 近畿 大学 は まだまだ 喰らいついて くる 。 ５００ メートル 地点 。 しんどい ・・・ 。 いつも より ペース が 速い 。 レース だ から 仕方ない けど ・・・ 。 近畿 大学 より は 少し 出た か ・・・ ？ でも まだ あと ５００ も ある 。 油断 は 禁物 。 ７５０ メートル 地点 。 近畿 大学 に 差 を 縮め られた か ・・・ ？ 並んで る 。 やばい ・・・ 。 ラスト スパート で ペース を 上げる も 、 漕ぎ が 小さく なって いて 艇 速 が 上がら ない ・・・ 。 ゴール 寸前 。 並んで る 。 横 一線 だ 。 これ は １ 秒 の 差 も ない くらい かも しれ ない 。 ファファッ ・・・ 。 どっち だ ？ 負けた ・・・ ？ 陸上 に 上がって 順位 を 聞く と ２ 位 。 しかも その 差 は ０．１７ 秒 。 日ごろ の ０．１７ 秒 なんて 何にも でき ない くらい の 短 さ 。 生まれて 初めて こんなに 悔しい もの は ない と 思った 。 ０．１７ 秒 の 重 み が ずっしり と 感じ られた 。 次の レース は ３ 週間 後 。 この 悔し さ 、 すべて ぶつけて やる 。,Sports
122,さて 、 こんな ブログ を 書いて いる 俺 も 京都 に 住んで いる わけです 。 京都 って の は 日本 有数 の 観光 地 らしく 、 友達 が たまーに 泊りがけ 

In [17]:
data_text_classification["label"].value_counts()

Kyoto      91
Keitai     79
Gourmet    57
Sports     22
Name: label, dtype: int64

In [18]:
split_train_dev_test(data_text_classification, "text_classification_KNBC", ("sentence", "label"))
print("完了!")

完了!


In [19]:
!wc -l text_classification_KNBC/*.tsv

   249 text_classification_KNBC/all.tsv
    26 text_classification_KNBC/dev.tsv
    26 text_classification_KNBC/test.tsv
   200 text_classification_KNBC/train.tsv
   501 total


ここまでは評判分析と全く同じです。

### 練習問題 1
ラベルが変わったことによりデータを読みこむ部分を追加しなければいけません。データの読みこみは transformers/transformers/data/processors/glue.py で行っています。このファイルを編集するにはファイル一覧の画面に移動して、ファイル名をクリックするとファイルを開くことができます。

SST-2は以下のクラス Sst2Processor でデータの読みこみをしています。これを真似して TextClassificationKNBCProcessor を作ってみましょう。317行目あたりにSst2Processorをコピーして TextClassificationKNBCProcessor を作っていますので、関数 get_labels のところだけを変更してください。

あとは同じファイルの末尾に、ラベルの数、どのプロセッサを使うのか、タスクが分類(classification)か回帰(regression)であるかを指定するところがあります。今回は時間の都合上、すでに以下のように追加しています。タスク名は tc-knbc としています(tcはtext classificationの意味)。

ファイルをsaveしてください。

あと、transformers/transformers/data/metrics/\_\_init\_\_.py で何を評価尺度として用いるかを指定しているところがあります。これも以下のようにすでに追加しています。

transformers/transformers以下のファイルを更新した場合はpipで再インストールする必要がありますので以下を実行してください。

In [20]:
!pip install transformers/

Processing ./transformers
  Stored in directory: /tmp/pip-ephem-wheel-cache-axxttb2z/wheels/7b/98/b9/2da18dcef55b090a377c480bc2c98287794672928a7a1e869e
Successfully built transformers
  Found existing installation: transformers 2.1.1
    Uninstalling transformers-2.1.1:
      Successfully uninstalled transformers-2.1.1


### 手順2. Fine-tuning

準備が整ったのでfine-tuningしましょう。--task_nameで"tc-knbc"を指定し、--output_dirで KNBC_result/text_classification/ を指定します。

In [21]:
!python ./transformers/examples/run_glue.py \
    --model_type bert \
    --model_name_or_path /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
    --task_name "tc-knbc" \
    --do_train \
    --do_eval \
    --save_steps 1000 \
    --data_dir text_classification_KNBC \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --output_dir KNBC_result/text_classification/ \
    --overwrite_output_dir \
    --overwrite_cache

11/28/2019 16:02:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/28/2019 16:02:58 - INFO - transformers.configuration_utils -   loading configuration file /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/config.json
11/28/2019 16:02:58 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "tc-knbc",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 32006
}

11/28/2019 16:02:58 - INFO - transfor

11/28/2019 16:03:04 - INFO - __main__ -   Saving features into cached file text_classification_KNBC/cached_train_Japanese_L-12_H-768_A-12_E-30_BPE_transformers_128_tc-knbc
11/28/2019 16:03:04 - INFO - __main__ -   ***** Running training *****
11/28/2019 16:03:04 - INFO - __main__ -     Num examples = 199
11/28/2019 16:03:04 - INFO - __main__ -     Num Epochs = 3
11/28/2019 16:03:04 - INFO - __main__ -     Instantaneous batch size per GPU = 8
11/28/2019 16:03:04 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 8
11/28/2019 16:03:04 - INFO - __main__ -     Gradient Accumulation steps = 1
11/28/2019 16:03:04 - INFO - __main__ -     Total optimization steps = 75
Epoch: 100%|██████████████████████████████████████| 3/3 [00:34<00:00, 11.62s/it]
11/28/2019 16:03:39 - INFO - __main__ -    global_step = 75, average loss = 0.5955811107158661
11/28/2019 16:03:39 - INFO - __main__ -   Saving model checkpoint to KNBC_result/text_classification/
11/28/2019 16

11/28/2019 16:03:42 - INFO - transformers.tokenization_utils -   Model name 'KNBC_result/text_classification/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased). Assuming 'KNBC_result/text_classification/' is a path or url to a directory containing tokenizer files.
11/28/2019 16:03:42 - INFO - transformers.tokenization_utils -   loading file KNBC_result/text_classification/vocab.txt
11/28/2019 16:03:42 - INFO - transformers.tokenization_utils -   loading file KNBC_result/text_classification/added_tokens.json
11/28/2019 16:03:42 - INFO - transformer

11/28/2019 16:03:45 - INFO - __main__ -   Saving features into cached file text_classification_KNBC/cached_dev_Japanese_L-12_H-768_A-12_E-30_BPE_transformers_128_tc-knbc
11/28/2019 16:03:45 - INFO - __main__ -   ***** Running evaluation  *****
11/28/2019 16:03:45 - INFO - __main__ -     Num examples = 25
11/28/2019 16:03:45 - INFO - __main__ -     Batch size = 8
Evaluating: 100%|█████████████████████████████████| 4/4 [00:00<00:00,  8.45it/s]
11/28/2019 16:03:45 - INFO - __main__ -   ***** Eval results  *****
11/28/2019 16:03:45 - INFO - __main__ -     acc = 0.96


3エポック回して約1分で終わります。4カテゴリが結構はっきり異なるジャンルなので90%前後と、高い精度で分類できました。

## 3. 固有表現抽出

最後に固有表現抽出 (Named Entity Recognition, NER) を行います。固有表現抽出とはテキスト中の人名、地名、組織名などの固有表現 (Named Entity)を抽出するタスクです。

これまでの講義でも説明されたとおり、固有表現抽出は系列ラベリングと呼ばれる手法で解かれることが多いです。以下の例で説明します。以下の文では「太郎」がPERSON、「京都大学」がORGANIZATIONです。「太郎」は1形態素ですが、「京都大学」は2形態素からなります。形態素が固有表現の場合、ラベルの頭にB (Begin)またはI (Inside)を付与し、固有表現でない場合、O (Outside)とし、各形態素のラベルを推定する問題となります。

単語 | 固有表現ラベル
--- | ---
太郎 | B-PERSON
は | O
京都 | B-ORGANIZATION
大学 | I-ORGANIZATION
に | O
行った | O

日本語の固有表現解析ではIREX (Information Retrieve and Extraction Exercise)で定義された、組織名(ORGANIZATION), 人名(PERSON), 地名(LOCATION), 固有物名(ARTIFACT), 日付表現(DATE), 時間表現(TIME), 金額表現(MONEY), 割合表現(PERCENT)の8種類を対象とすることが多いです。

### 手順1. 前処理

#記事単位で分割した方が良い。※記事を途中で分割しちゃうとテストデータの質がよろしくない。
以下のpythonスクリプトで固有表現データセットを生成します。

In [22]:
import glob
import os
import logging
import re
import random

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

random.seed(1)

corpus_types = [ "train", "dev", "test" ]
os.makedirs("ner_KNBC", exist_ok=True)
fs_out = { corpus_type: open(f"ner_KNBC/{corpus_type}.txt", "w") for corpus_type in corpus_types }

ne_pat = re.compile(r"<NE:(.+?):(.+?)>")

def get_corpus_type():
    # train:dev:test = 8:1:1 に split する
    rand = random.random()
    
    if rand >= 0.2:
        return "train"
    elif 0.1 <= rand < 0.2:
        return "dev"
    else:
        return "test"

corpus1_dirname = f"{KNBC_dir}/corpus1"

doc_num = 0
for dir in glob.glob(f"{corpus1_dirname}/*"):
    basename = os.path.basename(dir)
    corpus_type = get_corpus_type()

    f_out = fs_out[corpus_type]

    # for each sentence
    sentence_index = 1
    while (True):
        filename = f"{dir}/{basename}-1-{sentence_index}-01"
        if os.path.exists(filename) is False:
            break

        words, ne_labels = [], []
        with open(filename, "r", encoding="utf-8") as reader:
            buf = ""
            for line in reader.readlines():
                if line.startswith(("#", "*", "+", "EOS")):
                    continue
                word = (line.split(" "))[0]
                words.append(word)

                # 例：黒田 くろだ 黒田 名詞 6 人名 5 * 0 * 0 "疑似代表表記 代表表記:黒田/くろた" <疑似代表表記>..<NE:PERSON:head>
                m = ne_pat.search(line)
                if m:
                    category, position = m.groups()
                    if category == "OPTIONAL":
                        label = "O"
                    else:
                        if position == "head" or position == "single":
                            position_label = "B"
                        else:
                            position_label = "I"
                        label = f"{position_label}-{category}"
                else:
                    label = "O"
                ne_labels.append(label)

            for word, ne_label in zip(words, ne_labels):
                print(f"{word} {ne_label}", file=f_out)
            print(file=f_out)

        sentence_index += 1

for corpus_type in corpus_types:
    fs_out[corpus_type].close()

print("完了!")
!wc -l ner_KNBC/*

完了!
  4701 ner_KNBC/dev.txt
  8979 ner_KNBC/test.txt
 56460 ner_KNBC/train.txt
 70140 total


中身を見てみます。1カラム目が見出し、2カラム目が固有表現ラベルになっていて、空行が文区切りを表わします。

In [23]:
!head -n 50 ner_KNBC/dev.txt

［ O
京都 B-LOCATION
観光 O
］ O
二条城 B-LOCATION

二 O
週間 O
くらい O
前 O
、 O
友達 O
と O
二条城 B-LOCATION
に O
行って O
きた O
。 O

とにかく O
無駄に O
大きくて O
、 O
一 O
周 O
する O
の O
に O
結構な O
時間 O
が O
かかった O
し O
、 O
疲れた O
。 O

そこ O
で O
庭園 O
を O
見た O
んだ O
けど O
、 O
よく O
言わ O
れる O
ように O
ヨーロッパ B-LOCATION


### 練習問題 2
fine-tuningの前に、ラベル一覧を得ておく必要があります。1行に1ラベルとし、ner_KNBC/labels.txtに保存してください。

In [27]:
########## ここにコマンドを書いて下さい
!cat ner_KNBC/train.txt|grep -v "^$"|cut -d" " -f 2 |sort|uniq > ner_KNBC/labels.txt
!cat ner_KNBC/labels.txt
##########

B-ARTIFACT
B-DATE
B-LOCATION
B-MONEY
B-ORGANIZATION
B-PERCENT
B-PERSON
B-TIME
I-ARTIFACT
I-DATE
I-LOCATION
I-MONEY
I-ORGANIZATION
I-PERCENT
I-PERSON
I-TIME
O


### 手順2. Fine-tuning

系列ラベリングのfine-tuningは run_ner.py というスクリプトを使います。オプションはこれまでとほぼ同様です。--labelsオプションで上で生成したlabel一覧のファイルを指定するくらいが異なることです。

In [33]:
!python ./transformers/examples/run_ner.py --data_dir ./ner_KNBC/ \
--model_type bert \
--labels ./ner_KNBC/labels.txt \
--model_name_or_path /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
--output_dir KNBC_result/ner \
--max_seq_length 128 \
--num_train_epochs 3 \
#一気にn文を学習させる　多すぎるとメモリは足りなくなるが、多くしていった方が学習速度が速くなる
--per_gpu_train_batch_size 32 \
--save_steps 1000 \
--seed 1 \
--do_train \
--do_eval \
--do_predict \
--overwrite_output_dir \
--overwrite_cache

11/28/2019 16:38:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/28/2019 16:38:58 - INFO - transformers.configuration_utils -   loading configuration file /data/nlp/tool/bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/config.json
11/28/2019 16:38:58 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 17,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 32006
}

11/28/2019 16:38:58 - INFO - transformers

11/28/2019 16:39:06 - INFO - __main__ -   Saving features into cached file ./ner_KNBC/cached_train_Japanese_L-12_H-768_A-12_E-30_BPE_transformers_128
11/28/2019 16:39:07 - INFO - __main__ -   ***** Running training *****
11/28/2019 16:39:07 - INFO - __main__ -     Num examples = 3293
11/28/2019 16:39:07 - INFO - __main__ -     Num Epochs = 3
11/28/2019 16:39:07 - INFO - __main__ -     Instantaneous batch size per GPU = 32
11/28/2019 16:39:07 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 32
11/28/2019 16:39:07 - INFO - __main__ -     Gradient Accumulation steps = 1
11/28/2019 16:39:07 - INFO - __main__ -     Total optimization steps = 309
Iteration:   0%|                                        | 0/103 [00:00<?, ?it/s]/data/anaconda/envs/py36-tokyo-nlp-test/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them i

Iteration:  38%|███████████▋                   | 39/103 [00:54<01:29,  1.39s/it]^C


3エポックで10分弱かかります。この間に、GPU使用状況を確認できる nvidia-smi コマンドを使ってみましょう。
2日目にJuman++を使った時と同じように、New → Terminal で Terminal を開き、 `nvidia-smi` と打ってみてください。
重要なのは以下です。
* Volatile GPU-Util: GPUがどれくらい使われているか。100%に近いほどよい
* GPU Memory Usage: GPUメモリがどれくらい使われているか。他に制約がなければ最大に近いまで使うとよい

しばらく実行し続けるときは例えば `nvidia-smi -l 3` と打つと3秒ごとに実行されます。

最後に出ている数字が test での精度です。F値で 0.75 程度です。

システムの出力を簡単に確認してみましょう。KNBC_result/ner/test_predictions.txt がシステムの出力です。

In [29]:
!head -20 KNBC_result/ner/test_predictions.txt

［ O
京都 B-LOCATION
観光 O
］ O
京都 B-LOCATION
の O
バス O

昨日 B-DATE
、 O
久々に O
バス O
に O
乗り O
ました O
。 O

立って O
る O
人 O


### 練習問題 3*
上記の run_ner.py のオプション --per_gpu_train_batch_size はトレーニング時のバッチサイズを指定するものです。これは128や4にして実行してみてください。

### 手順3. 結果の可視化

システムの出力を検討する上で結果をわかりやすく表示することは非常に重要です。ここでは spacy というライブラリが提供している displacy を使って、固有表現解析の結果を可視化してみます。spacy はpipで簡単にインストールすることができます。(以下で使うtermcolorというライラブリも一緒にインストールしておきます)

In [34]:
!pip install spacy termcolor

displacy では以下のように文 (text)と固有表現の集合 (ents)を与えることによって、可視化することができます。「ents」のそれぞれの固有表現のstart/endはそれぞれ文頭からの文字数を表しています。(endは固有表現の末尾の文字位置 + 1)

In [35]:
import spacy
from spacy import displacy

ex = [{"text": "太郎 は 京都 大学 に 通っている 。",
       "ents": [{"start": 0, "end": 2, "label": "PERSON"},
                {"start": 5, "end": 10, "label": "ORG"}]}
      ]
displacy.render(ex, style="ent", manual=True, jupyter=True)

KNBCはコーパスサイズが小さかったためF値が0.75程度でしたが、固有表現解析で標準的に用いられているCRL固有表現データ(約1万文)ではF値0.92程度になります。古典的機械学習手法を用いた[笹野ら08]ではF値0.89と報告されていますので、固有表現解析でもBERTが強力であることがわかります。あらかじめ学習を走らせておき、その結果を /data/nlp/tool/bert/CRL 以下においていますので、これを用いて結果を可視化してみましょう。
**(これは毎日新聞のデータですので、持ち出さないようにお願いします)**

以下のpythonコードでシステムの出力と正解を可視化します。文単位でシステムの出力と正解が一致する場合はシステムの出力(=正解)を表示し、1文のどこかが異なる場合は上にシステムの出力、下に正解を表示します。

In [36]:
from seqeval.metrics.sequence_labeling import get_entities
import termcolor

tag_conversion_map = { "ORGANIZATION": "ORG",
                       "LOCATION": "LOC"}
class Word(object):
    def __init__(self, string, ner_tag, offset):
        self.string = string
        self.ner_tag = ner_tag
        self.start = offset
        self.end = offset + len(string)

def get_ner_example(lines):
    words = []
    offset = 0
    for line in lines:
        string, ner_tag = line.split(" ")
        word = Word(string, ner_tag, offset)
        words.append(word)
        # +1 は空白の分
        offset += len(word.string) + 1

    # [('PER', 0, 1), ('LOC', 3, 3)]
    entities = get_entities([ word.ner_tag for word in words ])
    spacy_entities = []
    for entity in entities:
        ner_label, start_word_index, end_word_index = entity 
        spacy_entities.append({ "start": words[start_word_index].start,
                                                    "end": words[end_word_index].end, 
                                                    "label": tag_conversion_map[ner_label] if ner_label in tag_conversion_map else ner_label })  
        
    return { "text": " ".join([ word.string for word in words ]),
                  "ents": spacy_entities }  

def get_ner_examples(filename):
    examples = []
    with open(filename, "r", encoding="utf-8") as reader:
        lines = [] 
        for line in reader.readlines():
            line = line.rstrip("\n")
      
            if line:
                lines.append(line)
            # 空行 (文の切れ目)
            else:
                example = get_ner_example(lines)
                examples.append(example)
                lines = []

    return examples

def is_same_example(system_example, gold_example):
    assert system_example["text"] == gold_example["text"]

    # entityの数が異なる
    if len(system_example["ents"]) != len(gold_example["ents"]):
        return False
    
    for system_ent, gold_ent in zip(system_example["ents"], gold_example["ents"]):
        # start, end, labelのいずれかが異なる
        if system_ent["start"] != gold_ent["start"] or system_ent["end"] != gold_ent["end"] or system_ent["label"] != gold_ent["label"]:
            return False

    # すべて一致したので全体が一致
        return True

def display_result(system_filename, gold_filename):
    system_examples = get_ner_examples(system_filename)
    gold_examples = get_ner_examples(gold_filename)

    for i, (system_example, gold_example) in enumerate(zip(system_examples, gold_examples)):
        if i == 100:
            break
        if is_same_example(system_example, gold_example) is True:
            displacy.render(system_example, style="ent", manual=True, jupyter=True)
        else:
            print("-" * 100)
            print(termcolor.colored("system", "blue"))
            displacy.render(system_example, style="ent", manual=True, jupyter=True)
            print(termcolor.colored("gold", "red"))
            displacy.render(gold_example, style="ent", manual=True, jupyter=True)
            print("-" * 100)
      
display_result("/data/nlp/tool/bert/CRL/test_predictions.txt", "/data/nlp/tool/bert/CRL/test.txt")

----------------------------------------------------------------------------------------------------
system


gold


----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
system


gold


...

gold


----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
system


gold


----------------------------------------------------------------------------------------------------


### 練習問題 4
上記の結果からどのようなことがわかるか分析してみよう。

*   システムはこんな難しいものでもわかるのか
*   逆にこんなやさしいものもわからないのか
*   システムがわからなくても仕方ない難しいもの
*   システムの出力の方が正しくて正解が間違っているのではないかというもの
*   タグ付けが誤っているのではないか


以上のように、BERTによるfine-tuningではプログラムの中身をほぼ変更することなく、入力データを用意するだけでなく、様々なタスクのfine-tuningを行うことができます。